In [8]:
import azure.ai.ml

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential =  DefaultAzureCredential()
    credential.get_token("https://management.azure.com/default")
except Exception as ex:
    print("i am here")
    credential = InteractiveBrowserCredential()
    



DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: No token received.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.


i am here


In [12]:
# i had to use cli credential with --no browser parameter , cause the redirect is having issues.

import azure.ai.ml

from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential


credential= AzureCliCredential()

In [13]:
ml_client = MLClient(
    credential=credential,
    subscription_id="31241946-ad24-4495-85e2-31248c60d164",
    resource_group_name="ariefinternal",
    workspace_name="ariefmlworkspace"
)

Register and upload the car dataset


In [14]:
#this uploads the file from the local store to the default blob store under the default container and then LocalUpload/4*******/good_car_data.csv
#this gibberish is a problem with the 4***
# and the studio shows this LocalUpload/4a337dba56504a0bc17f6e0e49d1adb1/good_car_data.csv 

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
my_path = "abfss://amex@ariefgen2.dfs.core.windows.net/data/mlcardata"
car_cleaned_data = Data(
    path="../../data/good_car_data.csv",
    type=AssetTypes.URI_FILE,
    description="Good Car Data",
    name="car",
    version='1'
)

ml_client.data.create_or_update(car_cleaned_data)

Uploading good_car_data.csv (< 1 MB): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44.2k/44.2k [00:00<00:00, 1.69MB/s]


Data({'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'car', 'description': 'Good Car Data', 'tags': {}, 'properties': {}, 'id': '/subscriptions/31241946-ad24-4495-85e2-31248c60d164/resourceGroups/ariefinternal/providers/Microsoft.MachineLearningServices/workspaces/ariefmlworkspace/data/car/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f06a811e640>, 'serialize': <msrest.serialization.Serializer object at 0x7f06ab6cfa90>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/31241946-ad24-4495-85e2-31248c60d164/resourcegroups/ariefinternal/workspaces/ariefmlworkspace/datastores/workspaceblobstore/paths/LocalUpload/4a337dba56504a0bc17f6e0e49d1adb1/good_car_data.csv', 'referenced_uris': None})

In [15]:
#this registers where the location is shown.
my_path = "abfss://amex@ariefgen2.dfs.core.windows.net/data/mlcardata/good_car_data.csv"
car_cleaned_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="The Real car Data",
    name="goodcardata",
    version='1'
)

ml_client.data.create_or_update(car_cleaned_data)

Data({'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'goodcardata', 'description': 'The Real car Data', 'tags': {}, 'properties': {}, 'id': '/subscriptions/31241946-ad24-4495-85e2-31248c60d164/resourceGroups/ariefinternal/providers/Microsoft.MachineLearningServices/workspaces/ariefmlworkspace/data/goodcardata/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f06a818e1c0>, 'serialize': <msrest.serialization.Serializer object at 0x7f06a815e8e0>, 'version': '1', 'latest_version': None, 'path': 'abfss://amex@ariefgen2.dfs.core.windows.net/data/mlcardata/good_car_data.csv', 'referenced_uris': None})

In [16]:
from azure.ai.ml.entities import AmlCompute
cpu_compute_target="amlcluster"

try:
    cpu_cluster =  ml_client.compute.get(cpu_compute_target)
    print(
        f"you already have a cluster named {cpu_compute_target}, we will resue it"
    )
except Exception:
    print("creating a new compute target")
    cpu_cluster=AmlCompute(
        name="amlcluster",
        type="aml-compute",
        size="STANDARD_D2_V2",
        min_instances=0,
        max_instances=2,
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated"
        )
    cpu_cluster=ml_client.begin_create_or_update(cpu_cluster)

print(
    f"AmlCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)


you already have a cluster named amlcluster, we will resue it
AmlCompute with name amlcluster is created, the compute size is STANDARD_D2_V2


In [17]:
import os
dependencies_dir="../dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [20]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scipy=1.7.1
  - scikit-learn=0.24.2
  - pandas>=1.1,<1.2
  - seaborn=0.11.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow==1.26.0
    - azureml-mlflow==1.41.0

    


Overwriting ../dependencies/conda.yml


In [21]:
from azure.ai.ml.entities import Environment
custom_env_name = "arief-scikit-learn"
pipeline_job_env = Environment(
    name=custom_env_name,
    description="arief scikit learn environment",
    tags={"scikit-learn":"0.24.2","whose-environment":"arief_scikit_env"},
    conda_file=os.path.join(dependencies_dir,"conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    version="1.0.0"

)

Creating the first Component

In [19]:
import os
data_prep_src_dir="../components/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)


In [ ]:
%%writefile {data_prep_src_dir}/data_prep_py

import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import mlflow
import seaborn

def main():
    """Main function of the script
    got it"""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, help="split ratio", required=False, default=0.2)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")

    args = parser.parse_args()

    #start logging
    mlflow.start_run()
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))
    print("input data:",args.data)


    car= pd.read_csv(args.data,header=1, index_col=0)
    print("initial size data", car.shape)
    
    #remove any price greater than 6000000
    car=car[car['Price']<6000000]
    print("Size after removing rows with price greater than 6000000 ", car.shape)

    








